In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from datasets import TripletAudio
from networks import AnchorNet, EmbeddingNet, TripletNet
from losses import TripletLoss
import torch.optim as optim
from recall import Recall
from torch.optim import lr_scheduler
from trainer import fit
from recall import Recall
import itertools
from tensorboardX import SummaryWriter

In [2]:
#define hyperparams
K, MAX_CLOSE_NEG, P_STRONG_NEG = 5, 15, 1
BATCH_SIZE = 128
INPUT_D, OUTPUT_D = 192, 128
MARGIN, N_EPOCHS, LOG_INT, N_RECALL_CAND = 0.5, 20, 100, 25
N_RECALL_SAMPLES = 1000
# hyperparams = [P_STRONG_NEGS]

In [12]:
#computes the recall on datasets using global variables such as model and recall
def measure_recall(datasets):
    outputs = []
    for d in datasets:
        batch_indicies = np.random.choice(list(range(0, d.get_dataset().shape[0])), N_RECALL_SAMPLES, False)
        queries = d.get_dataset()[batch_indicies]
        knns_of_batch = d.get_knn().iloc[batch_indicies, :]
        outputs.append(recall.calculate(d.get_dataset(), model.embedding_net, queries, knns_of_batch, False))
    return [np.mean(x) for x in outputs]

In [4]:
# while True:
#     for P_STRONG_NEG in P_STRONG_NEGS: #itertools.product(*hyperparams):
#setup datasets
triplet_train_dataset = TripletAudio(True, K, MAX_CLOSE_NEG, P_STRONG_NEG)
triplet_test_dataset = TripletAudio(False, K, MAX_CLOSE_NEG, P_STRONG_NEG)
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)
#define model 
anchor_net = AnchorNet(triplet_train_dataset.get_dataset(), INPUT_D, OUTPUT_D)
embedding_net = EmbeddingNet(anchor_net)
model = TripletNet(embedding_net)
loss_fn = TripletLoss(MARGIN)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1, last_epoch=-1)
recall = Recall(N_RECALL_CAND, K)
#run the model
train_loss, val_loss = fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT)
#measure recall
train_recall, val_recall = measure_recall([triplet_train_dataset, triplet_test_dataset])
#write to tensorboard
with SummaryWriter() as w:
    w.add_hparams(
        {'LR': LEARNING_RATE, 'BSIZE': BATCH_SIZE, 'N_RECALL_S': N_RECALL_SAMPLES, 'N_RECALL_CAND': N_RECALL_CAND,
             'CLOSE_NEG': MAX_CLOSE_NEG, 'P_STRONG_NEG': P_STRONG_NEG, 'OUTPUT_D': OUTPUT_D},
        {'TRAIN_L': train_loss, 'VAL_L': val_loss, 'TRAIN_RECALL': train_recall, 'VAL_RECALL': val_recall})

initialising model biases
done
Train: [0/48048 (0%)]	Loss: 11.901488
Train: [12800/48048 (27%)]	Loss: 16.377597
Train: [25600/48048 (53%)]	Loss: 15.947442
Train: [38400/48048 (80%)]	Loss: 15.238198
Epoch: 1/20. Train set: Average loss: 15.4714
Epoch: 1/20. Validation set: Average loss: 17.2634
Train: [0/48048 (0%)]	Loss: 14.845743
Train: [12800/48048 (27%)]	Loss: 14.133692
Train: [25600/48048 (53%)]	Loss: 11.837628
Train: [38400/48048 (80%)]	Loss: 11.312243
Epoch: 2/20. Train set: Average loss: 12.0827
Epoch: 2/20. Validation set: Average loss: 12.0750
Train: [0/48048 (0%)]	Loss: 14.880750
Train: [12800/48048 (27%)]	Loss: 9.495290
Train: [25600/48048 (53%)]	Loss: 8.550224
Train: [38400/48048 (80%)]	Loss: 8.116601
Epoch: 3/20. Train set: Average loss: 8.4474
Epoch: 3/20. Validation set: Average loss: 8.1786
Train: [0/48048 (0%)]	Loss: 11.137297
Train: [12800/48048 (27%)]	Loss: 6.804248
Train: [25600/48048 (53%)]	Loss: 6.237096
Train: [38400/48048 (80%)]	Loss: 5.315881
Epoch: 4/20. Train

ValueError: could not broadcast input array from shape (1000,3) into shape (1000)